<img src="GWO.jpg" alt="GWO" width=300>

#### Enhancing Person Identification via EEG Channel Selection Utilizing Binary Grey Wolf Optimization Algorithm
## 
## 


### Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix

### Load Data

In [2]:
def load_eeg_data():
    """
    Load EEG dataset from a CSV file.

    Returns:
        X (numpy.ndarray): Features (input data) from the EEG dataset.
        y (numpy.ndarray): Labels (output data) from the EEG dataset.
    """

    # Read the EEG dataset from the CSV file
    dataset = pd.read_csv("eeg_dataset.csv")

    # Extract features (X) and labels (y)
    X = dataset.drop("Label", axis=1).values
    y = dataset["Label"].values

    return X, y

### Multiclass Specificity Score

In [3]:
def specificity_score(y_true, y_pred):
    """
    Calculate specificity score for a multiclass classification task.

    Args:
        y_true (numpy.ndarray): True labels.
        y_pred (numpy.ndarray): Predicted labels.

    Returns:
        float: Mean specificity score across all classes.
    """
    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Initialize a list to store class-specific specificities
    class_specificities = []

    # Loop through each class
    for i in range(len(cm)):
        # True Negative (TN): sum of entries excluding current row and column
        tn = np.sum(np.delete(cm, i, axis=0)[:, np.delete(np.arange(len(cm)), i)])
        
        # False Positive (FP): sum of entries in the current column excluding the true positive
        fp = np.sum(np.delete(cm, i, axis=0)[:, i])
        
        # False Negative (FN): sum of entries in the current row excluding the true positive
        fn = np.sum(cm[i, np.delete(np.arange(len(cm)), i)])
        
        # True Positive (TP): entry in the current row and column
        tp = cm[i, i]
        
        # Specificity for the current class
        specificity = tn / (tn + fp)
        
        # Append the specificity to the list
        class_specificities.append(specificity)
    
    # Return the mean specificity across all classes
    return np.mean(class_specificities)

### Objective Function

In [4]:
def objective_function(selected_channels, X, y, clf, num_coefficients=5):
    """
    Objective function for EEG channel selection using BGWO-SVM.
    
    Args:
        selected_channels (numpy.ndarray): Boolean array indicating selected channels.
        X (numpy.ndarray): Input features, including all EEG channels.
        y (numpy.ndarray): True labels.
        clf: Classifier model (e.g., SVM with RBF kernel).
        num_coefficients (int): Number of coefficients to consider (default is 5).

    Returns:
        float: Negative combination of accuracy, F1-score, recall, and specificity.
    """
    # Filter selected channels based on the boolean array
    selected_channels = np.array(selected_channels, dtype=bool)
    X_selected = X[:, selected_channels]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    # Train the classifier
    clf.fit(X_train, y_train)

    # Predict on the test set
    y_pred = clf.predict(X_test)

    # Evaluate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted') 
    recall = recall_score(y_test, y_pred, average='weighted') 
    specificity = specificity_score(y_test, y_pred)

    # Combine metrics with a negative sign (as we aim to maximize the negative of the combination)
    return -(accuracy + f1 + recall + specificity)

### Binary Grey Wolf Optimizer (BGWO)

In [5]:
def binary_grey_wolf_optimizer(objective_func, population_size, num_channels, iterations, args=()):
    """
    Binary Grey Wolf Optimizer for channel selection using a given objective function.

    Args:
        objective_func (callable): Objective function to optimize.
        population_size (int): Size of the binary population.
        num_channels (int): Number of EEG channels.
        iterations (int): Number of iterations for optimization.
        args (tuple): Additional arguments for the objective function (default is an empty tuple).

    Returns:
        tuple: Tuple containing the best-selected channels and their corresponding performance score.
    """
    # Initialize alpha position and score
    alpha_position = np.zeros(num_channels)
    alpha_score = float('-inf')

    # Initialize the population
    population = initialize_population(population_size, num_channels)

    # Iterate through optimization iterations
    for iteration in range(iterations):
        # Evaluate fitness for each individual in the population
        for i in range(population_size):
            fitness = objective_func(population[i, :], *args)

            # Update alpha position if a better solution is found
            if fitness > alpha_score:
                alpha_score = fitness
                alpha_position = population[i, :].copy()

        # Update exploration-exploitation parameter 'a'
        a = 2 - 2 * iteration / iterations

        # Update each individual's position in the population
        for i in range(population_size):
            r1 = np.random.rand(num_channels)
            r2 = np.random.rand(num_channels)
            A1 = 2 * a * r1 - a
            C1 = 2 * r2

            # Update position based on Grey Wolf Optimization formula
            D_alpha = abs(C1 * alpha_position - population[i, :])
            X1 = (population[i, :] + alpha_position) / 2 + A1 * D_alpha / 2

            # Apply sigmoid function to update binary values
            population[i, :] = (np.random.rand(num_channels) < 1 / (1 + np.exp(-X1))).astype(bool)

    # Return the best-selected channels and their performance score
    return alpha_position, alpha_score

### Initialize Population

In [6]:
def initialize_population(population_size, num_channels):
    """
    Initialize a binary population for channel activation/deactivation.

    Args:
        population_size (int): Size of the binary population.
        num_channels (int): Number of EEG channels.

    Returns:
        numpy.ndarray: Binary population matrix with shape (population_size, num_channels).
    """
    # Generate a random binary matrix representing the activation/deactivation of EEG channels
    return np.random.randint(2, size=(population_size, num_channels), dtype=bool)

### Execution Script

In [7]:
if __name__ == "__main__":
    # Set the population size and number of iterations for the Binary Grey Wolf Optimizer
    population_size = 10
    iterations = 50

    # Load artificial EEG data
    X_artificial, y_artificial = load_eeg_data()

    # Initialize a Support Vector Machine (SVM) classifier with RBF kernel
    clf = SVC(kernel='rbf')

    # Determine the number of EEG channels based on the data
    num_channels = X_artificial.shape[1]

    # Run the Binary Grey Wolf Optimizer to select the best channels and evaluate performance
    best_channels, best_performance = binary_grey_wolf_optimizer(
        objective_function, population_size, num_channels, iterations, args=(X_artificial, y_artificial, clf)
    )

    # Print the results
    print(f"Best Selected Channels: {np.where(best_channels)[0]}")
    print(f"Best Performance: {best_performance}")

Best Selected Channels: [ 0  3  4  6  8  9 10 11 12]
Best Performance: -0.9432167935064611
